# MRI-QC - brainmask
https://mriqc.readthedocs.io/en/latest/docker.html#docker

## Test

In [ ]:
docker run -it nipreps/mriqc:latest --version

## Run the participant level in subjects 001 002 003

If the argument `--participant_label` is not provided, then all subjects will be processed and the group level analysis will automatically be executed without need of running the command in item 3.

Paths `<bids_dir>` and `<output_dir>` must be absolute. In particular, specifying relative paths for `<output_dir>` will generate no error and mriqc will run to completion without error but produce no output.

For security reasons, we recommend to run the docker command with the options `--read-only --tmpfs /run --tmpfs /tmp`. This will run the docker image in read-only mode, and map the temporary folders `/run` and `/tmp` to the temporal folder of the host.

In [ ]:
# default
docker run -it --rm -v <bids_dir>:/data:ro -v <output_dir>:/out nipreps/mriqc:latest /data /out participant --participant_label 001 002 003

# with my paths
docker run -it --rm -v /home/debi/jaime/repos/MR-EyeTrack/results/bids/derivatives/reconstructions:/data:ro -v /home/debi/jaime/repos/MR-EyeTrack/results/mriqc:/out nipreps/mriqc:latest /data /out participant --participant_label 001

# handle performance
docker run -it --rm \
-v /home/debi/jaime/repos/MR-EyeTrack/results/bids/derivatives/reconstructions:/data:ro \
-v /home/debi/jaime/repos/MR-EyeTrack/results/mriqc:/out \
-v /tmp/mriqc/work:/work \
nipreps/mriqc:latest \
/data /out \
participant \
--species human \
-m T1w \
--n_cpus 36 \
--ants-nthreads 9 \
--no-datalad-get \
--mem-gb 112 \
--no-sub \
--ants-settings /data/ants_settings.json \
--bids-filter-file /data/bids_filter.json \
--work-dir /work \
--dry-run

## Run the group level and report generation on previously processed (use the same `<output_dir>`) subjects

In [ ]:
# default
docker run -it --rm -v <bids_dir>:/data:ro -v <output_dir>:/out nipreps/mriqc:latest /data /out group #--read-only --tmpfs /run --tmpfs /tmp
# with my paths
docker run -it --rm -v /home/debi/jaime/repos/MR-EyeTrack/results/bids/derivatives/reconstructions:/data:ro -v /home/debi/jaime/repos/MR-EyeTrack/results/mriqc:/out nipreps/mriqc:latest /data /out group #--read-only --tmpfs /run --tmpfs /tmp

Loop of individual participants

In [ ]:
import os, subprocess

# def run_command(command):
#     process = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
#     console_output, console_errors = process.communicate()

def run_command(command):
    os.system(command)

input_folder = '/mnt/sda1/Repos/a-eye/Data/SHIP_dataset/non_labeled_dataset_bids'
output_folder = '/home/jaimebarranco/Desktop/MRI-QC/mriqc/mriqc_non-labeled-dataset'
n_procs = 12
n_threads = 12

n_sub = [f.path for f in os.scandir(input_folder) if f.is_dir() and f.name.startswith('sub-')] # number of subjects in input folder
n_sub = len(n_sub)
print(f'Number of subjects: {n_sub}')

# Run MRIQC for each subject
for sub in range(n_sub):
    # if sub-xxx_T1w.html exists, skip
    if os.path.isfile(f'{output_folder}/sub-{sub+1:03}_T1w.html'):
        print(f'Subject {sub+1:03} already processed')
        continue
    command = f'docker run --rm -v {input_folder}:/data:ro -v {output_folder}:/out nipreps/mriqc:latest /data /out participant --participant_label {sub+1:03} --nprocs {n_procs} --omp-nthreads {n_threads}'
    print(command)
    run_command(command)

# Run MRIQC group analysis
command = f'docker run --rm -v {input_folder}:/data:ro -v {output_folder}:/out nipreps/mriqc:latest /data /out group'
print(command)
run_command(command)

Check how many .html file reports were generated

In [ ]:
import os

input_folder = '/mnt/sda1/Repos/a-eye/Data/SHIP_dataset/non_labeled_dataset_bids'
output_folder = '/home/jaimebarranco/Desktop/MRI-QC/mriqc/mriqc_non-labeled-dataset'
n_sub = [f.path for f in os.scandir(input_folder) if f.is_dir() and f.name.startswith('sub-')] # number of subjects in input folder
n_sub = len(n_sub)

# check how many .html files are in output folder
n_html = len([f for f in os.listdir(output_folder) if f.endswith('.html')])
print(f'Number of .html files in output folder: {n_html}')

# which subjects have not been processed (range: 1-1210)
sub_list = [f for f in range(1, n_sub+1) if not os.path.isfile(f'{output_folder}/sub-{f:03}_T1w.html')]
print(f'Subjects not processed ({len(sub_list)}): {sub_list}')

# MREye-QC - brain/eye masks

## Subjective evaluation

In [ ]:
# mreye-qc
docker run -it --rm -v /home/jaimebarranco/Desktop/samples_v3_bids:/data:ro -v /home/jaimebarranco/Desktop/mreyeqc_output:/out mreyeqc:latest /data /out participant --participant_label 001 --nprocs 12 --omp-nthreads 12
# 7bd99c588704826d399898ec1f7419d40dc09b27473d511066f3cf8ab097fe5a

Loop

In [ ]:
import os, subprocess

# def run_command(command):
#     process = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
#     console_output, console_errors = process.communicate()

def run_command(command):
    os.system(command)

input_folder = '/home/jaimebarranco/Desktop/samples_v3_bids'
output_folder = '/home/jaimebarranco/Desktop/mreyeqc_output'
n_procs = 12
n_threads = 12

n_sub = [f.path for f in os.scandir(input_folder) if f.is_dir() and f.name.startswith('sub-')] # number of subjects in input folder
n_sub = len(n_sub)
print(f'Number of subjects: {n_sub}')

# Run MRIQC for each subject
for sub in range(1, 6):
    command = f'docker run --rm -v {input_folder}:/data:ro -v {output_folder}:/out mreyeqc_test:latest /data /out participant --participant_label {sub+1:03} --nprocs {n_procs} --omp-nthreads {n_threads}'
    print(command)
    run_command(command)


In [ ]:
# Run MRIQC group analysis
command = f'docker run --rm -v {input_folder}:/data:ro -v {output_folder}:/out mreyeqc_test:latest /data /out group'
print(command)
run_command(command)

# Analysis and plotting

Joint MRIQC {brain, eyes} masks

In [4]:
import pandas as pd

brain_tsv = '/home/debi/jaime/repos/MR-EyeTrack/results/mriqc/group_T1w.tsv'
eyes_tsv = '/home/debi/jaime/repos/MR-EyeTrack/results/mriqc_eyes/group_T1w.tsv'

# Read both TSV files
brain_df = pd.read_csv(brain_tsv, sep='\t')
eyes_df = pd.read_csv(eyes_tsv, sep='\t')

# Add a new column 'mask' to distinguish them
brain_df['mask'] = 'brain'
eyes_df['mask'] = 'eyes'

# Merge DataFrames
merged_df = pd.concat([brain_df, eyes_df], ignore_index=True)

# Extract 'position' and 'reconstruction' from 'bids_name'
merged_df['position'] = merged_df['bids_name'].str.extract(r'_acq-(up|down|left|right)_')[0]
merged_df['reconstruction'] = merged_df['bids_name'].str.extract(r'_rec-(steva[A-Za-z0-9]+)_T1w')[0]

# Keep only rec-steva* entries and extract reconstruction
merged_df = merged_df[merged_df['bids_name'].str.contains(r'_rec-steva', na=False)].reset_index(drop=True)

# Fill missing positions with None
merged_df['position'] = merged_df['position'].fillna(value="none")

# Check the result
print(merged_df[['bids_name', 'position', 'reconstruction', 'mask']].head())

                            bids_name position reconstruction   mask
0   sub-001_acq-down_rec-steva50p_T1w     down       steva50p  brain
1   sub-001_acq-down_rec-steva75p_T1w     down       steva75p  brain
2   sub-001_acq-down_rec-steva95p_T1w     down       steva95p  brain
3  sub-001_acq-down_rec-stevafull_T1w     down      stevafull  brain
4   sub-001_acq-left_rec-steva50p_T1w     left       steva50p  brain


Merge DCTE results into the MRIQC merged_df

In [5]:
# Adds columns: dcte_csf, dcte_gm, dcte_wm, dcte (for Brain region cases) and dcte for Eyes
import os
import pandas as pd

# Paths
DCTE_CSV = os.path.join(os.getcwd(), "/home/debi/jaime/repos/MR-EyeTrack/analysis/comparison/dcte/dcte_all_combined.csv")
OUT_CSV = "/home/debi/jaime/repos/MR-EyeTrack/analysis/comparison/dcte_mriqc_combined.csv"

if 'merged_df' not in globals():
    raise RuntimeError("merged_df not found in the notebook environment. Run the earlier cell that builds merged_df before running this cell.")

if not os.path.exists(DCTE_CSV):
    raise FileNotFoundError(f"DCTE CSV not found: {DCTE_CSV}")

df_dcte = pd.read_csv(DCTE_CSV)
print(f"Loaded DCTE rows: {len(df_dcte)}")

# Remove volume mask rows as requested
df_dcte = df_dcte[df_dcte['mask'] != 'volume'].copy()
print(f"DCTE rows after dropping volume mask: {len(df_dcte)}")

# Pivot brain regions into columns (rename brain-region dcte to dcte_brain to avoid confusion)
df_brain = df_dcte[df_dcte['mask'] == 'brain'].copy()
if df_brain.empty:
    print("Warning: no brain mask rows found in DCTE file")

pivot = df_brain.pivot_table(index=['subject', 'position', 'reconstruction'],
                             columns='region',
                             values='dcte',
                             aggfunc='first').reset_index()

# Rename pivot columns to requested names (use dcte_brain for the Brain region)
col_map = {'CSF': 'dcte_csf', 'GM': 'dcte_gm', 'WM': 'dcte_wm', 'Brain': 'dcte_brain'}
for src, dst in col_map.items():
    if src in pivot.columns:
        pivot = pivot.rename(columns={src: dst})
    else:
        pivot[dst] = pd.NA

pivot = pivot[['subject', 'position', 'reconstruction', 'dcte_csf', 'dcte_gm', 'dcte_wm', 'dcte_brain']]
print(f"Pivot rows (brain): {len(pivot)}")

# Also extract eyes DCTE (only provides a single dcte per row)
df_eyes = df_dcte[df_dcte['mask'] == 'eyes'][['subject', 'position', 'reconstruction', 'dcte']].copy()
df_eyes = df_eyes.rename(columns={'dcte': 'dcte_eyes'})
print(f"Eyes rows in DCTE: {len(df_eyes)}")

# Prepare merged_df keys
if 'subject' not in merged_df.columns:
    merged_df['subject'] = merged_df['bids_name'].str.extract(r'^(sub-[0-9]+)')

# Map merged_df reconstruction values to DCTE labels:
merged_df['dcte_reconstruction'] = (
    merged_df['reconstruction'].astype(str).str.lower().str.replace('steva', '', regex=False)
)
mask_stevafull_noacq = (merged_df['position'].fillna('none') == 'none') & (merged_df['reconstruction'].astype(str).str.lower().str.contains('stevafull', na=False))
merged_df.loc[mask_stevafull_noacq, 'dcte_reconstruction'] = 'woBin'

# Merge brain pivot
merged = merged_df.merge(
    pivot,
    left_on=['subject', 'position', 'dcte_reconstruction'],
    right_on=['subject', 'position', 'reconstruction'],
    how='left',
    suffixes=('', '_dcte_brain')
)

# Merge eyes DCTE
merged = merged.merge(
    df_eyes,
    left_on=['subject', 'position', 'dcte_reconstruction'],
    right_on=['subject', 'position', 'reconstruction'],
    how='left',
    suffixes=('', '_dcte_eyes')
)

# Build final dcte column: use brain dcte for mask=='brain', eyes dcte for mask=='eyes'
merged['dcte'] = pd.NA
if 'dcte_brain' in merged.columns:
    merged.loc[merged['mask'].fillna('').astype(str) == 'brain', 'dcte'] = merged.loc[merged['mask'].fillna('').astype(str) == 'brain', 'dcte_brain']
if 'dcte_eyes' in merged.columns:
    merged.loc[merged['mask'].fillna('').astype(str) == 'eyes', 'dcte'] = merged.loc[merged['mask'].fillna('').astype(str) == 'eyes', 'dcte_eyes']

# Set the reconstruction column in the merged output to the DCTE label (keep consistent)
merged['reconstruction'] = merged['dcte_reconstruction']

# Drop helper columns from the right-hand sides and intermediate cols
cols_to_drop = ['dcte_reconstruction', 'reconstruction_dcte', 'reconstruction_dcte_brain', 'reconstruction_dcte_eyes', 'dcte_brain', 'dcte_eyes']
merged = merged.drop(columns=[c for c in cols_to_drop if c in merged.columns], errors='ignore')

# Ensure brain-only DCTE columns are only populated for rows with mask=='brain'
brain_cols = ['dcte_csf', 'dcte_gm', 'dcte_wm']
if 'mask' in merged.columns:
    non_brain_mask = merged['mask'].fillna('').astype(str) != 'brain'
    for c in brain_cols:
        if c in merged.columns:
            merged.loc[non_brain_mask, c] = pd.NA

# Ensure the final dcte is NA for non brain/eyes masks
if 'mask' in merged.columns:
    valid_mask = merged['mask'].fillna('').astype(str).isin(['brain', 'eyes'])
    merged.loc[~valid_mask, 'dcte'] = pd.NA

# Ensure output folder exists and save
# os.makedirs(os.path.dirname(OUT_CSV), exist_ok=True)
# merged.to_csv(OUT_CSV, index=False)
print(f"Merged DCTE+MRIQC saved to: {OUT_CSV}")

print(merged[['bids_name', 'subject', 'position', 'reconstruction', 'dcte_csf', 'dcte_gm', 'dcte_wm', 'dcte']].head())


Loaded DCTE rows: 306
DCTE rows after dropping volume mask: 255
Pivot rows (brain): 51
Eyes rows in DCTE: 51
Merged DCTE+MRIQC saved to: /home/debi/jaime/repos/MR-EyeTrack/analysis/comparison/dcte_mriqc_combined.csv
                            bids_name  subject position reconstruction  \
0   sub-001_acq-down_rec-steva50p_T1w  sub-001     down            50p   
1   sub-001_acq-down_rec-steva75p_T1w  sub-001     down            75p   
2   sub-001_acq-down_rec-steva95p_T1w  sub-001     down            95p   
3  sub-001_acq-down_rec-stevafull_T1w  sub-001     down           full   
4   sub-001_acq-left_rec-steva50p_T1w  sub-001     left            50p   

    dcte_csf    dcte_gm    dcte_wm      dcte  
0  48.799804  27.545256  11.724280   2.47158  
1  52.391769  29.522829  12.588693  2.514747  
2  41.611364  24.182781  10.364960  1.993474  
3  48.923341  27.845902  11.863429  2.549106  
4  48.475011  28.426839  12.070848  2.544496  


Metrics
| Category | Metric |
|---|---|
| Image clarity | efc |
| Artifact level | qi1 |
| Bias field | inu_med |
| Contrast quality | cjv |
| Tissue separability | cnr |

## Boxplots

DCTE

In [6]:
# Boxplot from merged DCTE+MRIQC ('merged' produced in mri_qc.ipynb)
import pandas as pd
import plotly.express as px

# Path to the merged CSV produced by the MRIQC notebook
csv_path = "/home/debi/jaime/repos/MR-EyeTrack/analysis/comparison/dcte_mriqc_combined.csv"
df = pd.read_csv(csv_path)

# We need to expand brain rows (which have separate dcte_csf/dcte_gm/dcte_wm/dcte columns)
records = []
for _, r in df.iterrows():
    mask = r.get('mask') if 'mask' in r.index else None
    subj = r.get('subject') if 'subject' in r.index else None
    pos = r.get('position') if 'position' in r.index else None
    rec = r.get('reconstruction') if 'reconstruction' in r.index else None
    bids = r.get('bids_name') if 'bids_name' in r.index else None

    if str(mask) == 'brain':
        # create one row per tissue using the dcte_* columns
        tissue_map = [('dcte_csf','CSF'), ('dcte_gm','GM'), ('dcte_wm','WM'), ('dcte','Brain')]
        for col, tissue in tissue_map:
            val = r.get(col) if col in r.index else pd.NA
            records.append({
                'subject': subj,
                'position': pos,
                'reconstruction': rec,
                'mask': 'brain',
                'region': tissue,
                'mask_region': f'brain_{tissue}',
                'mask_region_label': tissue,
                'dcte': val,
                'bids_name': bids
            })
    else:
        # non-brain rows (e.g. eyes or volume) keep the single dcte column
        val = r.get('dcte') if 'dcte' in r.index else pd.NA
        label = 'Eyes' if str(mask) == 'eyes' else ('Whole Volume' if str(mask) == 'volume' else mask)
        records.append({
            'subject': subj,
            'position': pos,
            'reconstruction': rec,
            'mask': mask,
            'region': label,
            'mask_region': mask,
            'mask_region_label': label,
            'dcte': val,
            'bids_name': bids
        })

plot_df = pd.DataFrame.from_records(records)
# Drop rows without a valid DCTE value
plot_df = plot_df.dropna(subset=['dcte'])

# Ensure categorical ordering for plotting
recon_order = ['woBin', 'full', '50p', '75p', '95p']
mask_order = ['Whole Volume', 'Brain', 'CSF', 'GM', 'WM', 'Eyes']
position_order = ['none', 'up', 'down', 'left', 'right']
plot_df['reconstruction'] = pd.Categorical(plot_df['reconstruction'], categories=recon_order, ordered=True)
plot_df['mask_region_label'] = pd.Categorical(plot_df['mask_region_label'], categories=mask_order, ordered=True)
plot_df['position'] = pd.Categorical(plot_df['position'], categories=position_order, ordered=True)

# Draw boxplot: mask/region on x-axis, colored by reconstruction, faceted by position
fig = px.box(
    plot_df,
    x='mask_region_label',
    y='dcte',
    color='reconstruction',
    facet_col='position',
    points='all',
    title='DCTE by Mask Type and Position',
    category_orders={
        'mask_region_label': mask_order,
        'reconstruction': recon_order,
        'position': position_order
    }
)
fig.update_layout(boxmode='group')
fig.show()


In [7]:
import pandas as pd
import plotly.express as px

# Path to the merged CSV produced by the MRIQC notebook
csv_path = "/home/debi/jaime/repos/MR-EyeTrack/analysis/comparison/dcte_mriqc_combined.csv"
df = pd.read_csv(csv_path)

# Expand brain rows (dcte_csf/dcte_gm/dcte_wm/dcte)
records = []
for _, r in df.iterrows():
    mask = r.get('mask')
    subj = r.get('subject')
    pos = r.get('position')
    rec = r.get('reconstruction')
    bids = r.get('bids_name')

    if str(mask) == 'brain':
        tissue_map = [('dcte_csf','CSF'), ('dcte_gm','GM'), ('dcte_wm','WM'), ('dcte','Brain')]
        for col, tissue in tissue_map:
            val = r.get(col) if col in r.index else pd.NA
            records.append({
                'subject': subj,
                'position': pos,
                'reconstruction': rec,
                'mask': 'brain',
                'region': tissue,
                'mask_region': f'brain_{tissue}',
                'mask_region_label': tissue,
                'dcte': val,
                'bids_name': bids
            })
    else:
        val = r.get('dcte') if 'dcte' in r.index else pd.NA
        label = 'Eyes' if str(mask) == 'eyes' else ('Whole Volume' if str(mask) == 'volume' else mask)
        records.append({
            'subject': subj,
            'position': pos,
            'reconstruction': rec,
            'mask': mask,
            'region': label,
            'mask_region': mask,
            'mask_region_label': label,
            'dcte': val,
            'bids_name': bids
        })

plot_df = pd.DataFrame.from_records(records)
plot_df = plot_df.dropna(subset=['dcte'])

# Compute mean DCTE per subject across positions
df_mean = plot_df.groupby(
    ['subject', 'mask_region_label', 'reconstruction'], as_index=False
)['dcte'].mean()

# Ensure categorical ordering
recon_order = ['woBin', 'full', '50p', '75p', '95p']
mask_order = ['Whole Volume', 'Brain', 'CSF', 'GM', 'WM', 'Eyes']
df_mean['reconstruction'] = pd.Categorical(df_mean['reconstruction'], categories=recon_order, ordered=True)
df_mean['mask_region_label'] = pd.Categorical(df_mean['mask_region_label'], categories=mask_order, ordered=True)

# Draw boxplot of averaged DCTE per subject
fig_avg = px.box(
    df_mean,
    x='mask_region_label',
    y='dcte',
    color='reconstruction',
    points='all',
    title='Mean DCTE per Subject across Positions',
    category_orders={
        'mask_region_label': mask_order,
        'reconstruction': recon_order
    }
)
fig_avg.update_layout(boxmode='group')
fig_avg.show()


SNR

In [8]:
# Build SNR-long dataframe from `merged` (exclude 'eyes' mask)
records_snr = []
for _, r in merged.iterrows():
    mask = r.get('mask')
    # skip eyes mask
    if str(mask).lower() == 'eyes':
        continue

    subj = r.get('subject')
    pos = r.get('position')
    rec = r.get('reconstruction')
    bids = r.get('bids_name')

    if str(mask) == 'brain':
        tissue_map = [('snr_csf', 'CSF'), ('snr_gm', 'GM'), ('snr_wm', 'WM'), ('snr_total', 'Brain')]
        for col, tissue in tissue_map:
            val = r.get(col) if col in r.index else pd.NA
            records_snr.append({
                'subject': subj,
                'position': pos,
                'reconstruction': rec,
                'mask': 'brain',
                'region': tissue,
                'mask_region': f'brain_{tissue}',
                'mask_region_label': tissue,
                'snr': val,
                'bids_name': bids
            })
    else:
        val = r.get('snr_total') if 'snr_total' in r.index else pd.NA
        label = 'Whole Volume' if str(mask) == 'volume' else mask
        records_snr.append({
            'subject': subj,
            'position': pos,
            'reconstruction': rec,
            'mask': mask,
            'region': label,
            'mask_region': mask,
            'mask_region_label': label,
            'snr': val,
            'bids_name': bids
        })

plot_snr = pd.DataFrame.from_records(records_snr)
plot_snr = plot_snr.dropna(subset=['snr'])

# Keep same categorical ordering as DCTE plot
plot_snr['reconstruction'] = pd.Categorical(plot_snr['reconstruction'], categories=recon_order, ordered=True)
plot_snr['mask_region_label'] = pd.Categorical(plot_snr['mask_region_label'], categories=mask_order, ordered=True)

# Boxplot of SNR
fig = px.box(
    plot_snr,
    x='mask_region_label',
    y='snr',
    color='reconstruction',
    facet_col='position',
    points='all',
    title='SNR by Mask Type and Position (eyes removed)',
    category_orders={'mask_region_label': mask_order, 'reconstruction': recon_order, 'position': position_order}
)
fig.update_layout(boxmode='group')
fig.show()


SNR mean

In [9]:
import pandas as pd
import plotly.express as px

# Build SNR-long dataframe from `merged` (exclude 'eyes' mask)
records_snr = []
for _, r in merged.iterrows():
    mask = r.get('mask')
    if str(mask).lower() == 'eyes':
        continue

    subj = r.get('subject')
    pos = r.get('position')
    rec = r.get('reconstruction')
    bids = r.get('bids_name')

    if str(mask) == 'brain':
        tissue_map = [('snr_csf', 'CSF'), ('snr_gm', 'GM'), ('snr_wm', 'WM'), ('snr_total', 'Brain')]
        for col, tissue in tissue_map:
            val = r.get(col) if col in r.index else pd.NA
            records_snr.append({
                'subject': subj,
                'position': pos,
                'reconstruction': rec,
                'mask': 'brain',
                'region': tissue,
                'mask_region': f'brain_{tissue}',
                'mask_region_label': tissue,
                'snr': val,
                'bids_name': bids
            })
    else:
        val = r.get('snr_total') if 'snr_total' in r.index else pd.NA
        label = 'Whole Volume' if str(mask) == 'volume' else mask
        records_snr.append({
            'subject': subj,
            'position': pos,
            'reconstruction': rec,
            'mask': mask,
            'region': label,
            'mask_region': mask,
            'mask_region_label': label,
            'snr': val,
            'bids_name': bids
        })

plot_snr = pd.DataFrame.from_records(records_snr)
plot_snr = plot_snr.dropna(subset=['snr'])

# Compute mean SNR per subject across positions
df_snr_mean = plot_snr.groupby(
    ['subject', 'mask_region_label', 'reconstruction'], as_index=False
)['snr'].mean()

# Ensure categorical ordering (same as DCTE plot)
recon_order = ['woBin', 'full', '50p', '75p', '95p']
mask_order = ['Whole Volume', 'Brain', 'CSF', 'GM', 'WM', 'Eyes']  # eyes are excluded but keep ordering
df_snr_mean['reconstruction'] = pd.Categorical(df_snr_mean['reconstruction'], categories=recon_order, ordered=True)
df_snr_mean['mask_region_label'] = pd.Categorical(df_snr_mean['mask_region_label'], categories=mask_order, ordered=True)

# Boxplot of averaged SNR per subject
fig_avg = px.box(
    df_snr_mean,
    x='mask_region_label',
    y='snr',
    color='reconstruction',
    points='all',
    title='Mean SNR per Subject across Positions',
    category_orders={'mask_region_label': mask_order, 'reconstruction': recon_order}
)
fig_avg.update_layout(boxmode='group')
fig_avg.show()


SNRd

https://mriqc.readthedocs.io/en/latest/iqms/t1w.html#mriqc.qc.anatomical.snr_dietrich

SNR using background air as reference

In [10]:
# Build SNRd-long dataframe from `merged` (exclude 'eyes' mask)
records_snrd = []
for _, r in merged.iterrows():
    mask = r.get('mask')
    if str(mask).lower() == 'eyes':
        continue

    subj = r.get('subject')
    pos = r.get('position')
    rec = r.get('reconstruction')
    bids = r.get('bids_name')

    if str(mask) == 'brain':
        tissue_map = [('snrd_csf', 'CSF'), ('snrd_gm', 'GM'), ('snrd_wm', 'WM'), ('snrd_total', 'Brain')]
        for col, tissue in tissue_map:
            val = r.get(col) if col in r.index else pd.NA
            records_snrd.append({
                'subject': subj,
                'position': pos,
                'reconstruction': rec,
                'mask': 'brain',
                'region': tissue,
                'mask_region': f'brain_{tissue}',
                'mask_region_label': tissue,
                'snrd': val,
                'bids_name': bids
            })
    else:
        val = r.get('snrd_total') if 'snrd_total' in r.index else pd.NA
        label = 'Whole Volume' if str(mask) == 'volume' else mask
        records_snrd.append({
            'subject': subj,
            'position': pos,
            'reconstruction': rec,
            'mask': mask,
            'region': label,
            'mask_region': mask,
            'mask_region_label': label,
            'snrd': val,
            'bids_name': bids
        })

plot_snrd = pd.DataFrame.from_records(records_snrd)
plot_snrd = plot_snrd.dropna(subset=['snrd'])

# Keep same categorical ordering as DCTE plot, but remove 'Eyes' from mask order
try:
    mask_order_snrd = [m for m in mask_order if m != 'Eyes']
except NameError:
    mask_order_snrd = ['Whole Volume', 'Brain', 'CSF', 'GM', 'WM']

plot_snrd['reconstruction'] = pd.Categorical(plot_snrd['reconstruction'], categories=recon_order, ordered=True)
plot_snrd['mask_region_label'] = pd.Categorical(plot_snrd['mask_region_label'], categories=mask_order_snrd, ordered=True)
plot_snrd['position'] = pd.Categorical(plot_snrd['position'], categories=position_order, ordered=True)

# Boxplot of SNRd (eyes removed)
fig = px.box(
    plot_snrd,
    x='mask_region_label',
    y='snrd',
    color='reconstruction',
    facet_col='position',
    points='all',
    title='SNRd by Mask Type and Position (eyes removed)',
    category_orders={'mask_region_label': mask_order_snrd, 'reconstruction': recon_order, 'position': position_order}
)
fig.update_layout(boxmode='group')
fig.show()


SNRd mean

In [11]:
import pandas as pd
import plotly.express as px

# Assuming 'merged' dataframe is already loaded

# Build SNRd-long dataframe from `merged` (exclude 'eyes' mask)
records_snrd = []
for _, r in merged.iterrows():
    mask = r.get('mask')
    if str(mask).lower() == 'eyes':
        continue

    subj = r.get('subject')
    pos = r.get('position')
    rec = r.get('reconstruction')
    bids = r.get('bids_name')

    if str(mask) == 'brain':
        tissue_map = [('snrd_csf', 'CSF'), ('snrd_gm', 'GM'), ('snrd_wm', 'WM'), ('snrd_total', 'Brain')]
        for col, tissue in tissue_map:
            val = r.get(col) if col in r.index else pd.NA
            records_snrd.append({
                'subject': subj,
                'position': pos,
                'reconstruction': rec,
                'mask': 'brain',
                'region': tissue,
                'mask_region': f'brain_{tissue}',
                'mask_region_label': tissue,
                'snrd': val,
                'bids_name': bids
            })
    else:
        val = r.get('snrd_total') if 'snrd_total' in r.index else pd.NA
        label = 'Whole Volume' if str(mask) == 'volume' else mask
        records_snrd.append({
            'subject': subj,
            'position': pos,
            'reconstruction': rec,
            'mask': mask,
            'region': label,
            'mask_region': mask,
            'mask_region_label': label,
            'snrd': val,
            'bids_name': bids
        })

plot_snrd = pd.DataFrame.from_records(records_snrd)
plot_snrd = plot_snrd.dropna(subset=['snrd'])

# Compute mean SNRd per subject across positions
df_snrd_mean = plot_snrd.groupby(
    ['subject', 'mask_region_label', 'reconstruction'], as_index=False
)['snrd'].mean()

# Ensure categorical ordering (remove 'Eyes')
mask_order_snrd = ['Whole Volume', 'Brain', 'CSF', 'GM', 'WM']
recon_order = ['woBin', 'full', '50p', '75p', '95p']

df_snrd_mean['reconstruction'] = pd.Categorical(df_snrd_mean['reconstruction'], categories=recon_order, ordered=True)
df_snrd_mean['mask_region_label'] = pd.Categorical(df_snrd_mean['mask_region_label'], categories=mask_order_snrd, ordered=True)

# Boxplot of averaged SNRd per subject
fig_avg = px.box(
    df_snrd_mean,
    x='mask_region_label',
    y='snrd',
    color='reconstruction',
    points='all',
    title='Mean SNRd per Subject across Positions',
    category_orders={'mask_region_label': mask_order_snrd, 'reconstruction': recon_order}
)
fig_avg.update_layout(boxmode='group')
fig_avg.show()


In [12]:
# create a dataframe with snrd_total for masks brain and eyes from merged dataframe to see them together in two separate columns
snrd_brain = merged[merged['mask'] == 'brain'][['subject', 'position', 'reconstruction', 'snrd_total']]
snrd_brain = snrd_brain.rename(columns={'snrd_total': 'snrd_brain'})
snrd_eyes = merged[merged['mask'] == 'eyes'][['subject', 'position', 'reconstruction', 'snrd_total']]
snrd_eyes = snrd_eyes.rename(columns={'snrd_total': 'snrd_eyes'})  
snrd_combined = pd.merge(snrd_brain, snrd_eyes, on=['subject', 'position', 'reconstruction'], how='inner')
print(snrd_combined.head())

   subject position reconstruction  snrd_brain  snrd_eyes
0  sub-001     down            50p   44.990281  50.116948
1  sub-001     down            75p   51.295304  51.148009
2  sub-001     down            95p   50.098536  50.393125
3  sub-001     down           full   49.988065  44.892085
4  sub-001     left            50p   49.767123  48.680827


INU mean

https://mriqc.readthedocs.io/en/latest/iqms/t1w.html#measures-targeting-specific-artifacts

Summary statistics (max, min and median) of the INU field (bias field) as extracted by the N4ITK algorithm [Tustison2010]. Values closer to 1.0 are better, values further from zero indicate greater RF field inhomogeneity.

In [13]:
import pandas as pd
import plotly.express as px

# Assuming 'merged' dataframe is already loaded

# Build inu_med-long dataframe (brain-only, eyes removed)
records_inu = []
for _, r in merged.iterrows():
    mask = str(r.get('mask')).lower() if r.get('mask') is not None else ''
    if mask != 'brain':
        continue
    val = r.get('inu_med') if 'inu_med' in r.index else pd.NA
    records_inu.append({
        'subject': r.get('subject'),
        'position': r.get('position'),
        'reconstruction': r.get('reconstruction'),
        'mask': mask,
        'mask_region_label': 'Brain',
        'inu_med': val,
        'bids_name': r.get('bids_name')
    })

plot_inu = pd.DataFrame.from_records(records_inu)
plot_inu = plot_inu.dropna(subset=['inu_med'])

# Compute mean inu_med per subject across positions
df_inu_mean = plot_inu.groupby(
    ['subject', 'mask_region_label', 'reconstruction'], as_index=False
)['inu_med'].mean()

# Ensure categorical ordering
recon_order = ['woBin', 'full', '50p', '75p', '95p']
df_inu_mean['reconstruction'] = pd.Categorical(df_inu_mean['reconstruction'], categories=recon_order, ordered=True)
df_inu_mean['mask_region_label'] = pd.Categorical(df_inu_mean['mask_region_label'], categories=['Brain'], ordered=True)

# Boxplot of averaged inu_med per subject
fig_avg = px.box(
    df_inu_mean,
    x='mask_region_label',
    y='inu_med',
    color='reconstruction',
    points='all',
    title='Mean INU (Brain) per Subject across Positions',
    category_orders={'mask_region_label': ['Brain'], 'reconstruction': recon_order}
)
fig_avg.update_layout(boxmode='group')
fig_avg.show()


EFC

https://mriqc.readthedocs.io/en/latest/iqms/t1w.html#mriqc.qc.anatomical.efc

Entropy focus criterion (image entropy). Calculate the EFC [Atkinson1997]. Uses the Shannon entropy of voxel intensities as an indication of ghosting and blurring induced by head motion. A range of low values is better, with EFC = 0 for all the energy concentrated in one pixel.

In [14]:
# Build efc-long dataframe from `merged` (brain-only, eyes removed)
records_efc = []
for _, r in merged.iterrows():
    mask = str(r.get('mask')).lower() if r.get('mask') is not None else ''
    if mask != 'brain':
        continue
    val = r.get('efc') if 'efc' in r.index else pd.NA
    records_efc.append({
        'subject': r.get('subject'),
        'position': r.get('position'),
        'reconstruction': r.get('reconstruction'),
        'mask': mask,
        'mask_region_label': 'Brain',
        'efc': val,
        'bids_name': r.get('bids_name')
    })

plot_efc = pd.DataFrame.from_records(records_efc)
plot_efc = plot_efc.dropna(subset=['efc'])

# Ensure categorical ordering (reuse recon_order and position_order defined earlier)
plot_efc['reconstruction'] = pd.Categorical(plot_efc['reconstruction'], categories=recon_order, ordered=True)
plot_efc['mask_region_label'] = pd.Categorical(plot_efc['mask_region_label'], categories=['Brain'], ordered=True)
plot_efc['position'] = pd.Categorical(plot_efc['position'], categories=position_order, ordered=True)

# Boxplot of EFC (eyes removed)
fig = px.box(
    plot_efc,
    x='mask_region_label',
    y='efc',
    color='reconstruction',
    facet_col='position',
    points='all',
    title='EFC by Mask Type and Position (eyes removed)',
    category_orders={
        'mask_region_label': ['Brain'],
        'reconstruction': recon_order,
        'position': position_order
    }
)
fig.update_layout(boxmode='group')
fig.show()


EFC mean

In [15]:
import pandas as pd
import plotly.express as px

# Assuming 'merged' dataframe is already loaded

# Build efc-long dataframe (brain-only, eyes removed)
records_efc = []
for _, r in merged.iterrows():
    mask = str(r.get('mask')).lower() if r.get('mask') is not None else ''
    if mask != 'brain':
        continue
    val = r.get('efc') if 'efc' in r.index else pd.NA
    records_efc.append({
        'subject': r.get('subject'),
        'position': r.get('position'),
        'reconstruction': r.get('reconstruction'),
        'mask': mask,
        'mask_region_label': 'Brain',
        'efc': val,
        'bids_name': r.get('bids_name')
    })

plot_efc = pd.DataFrame.from_records(records_efc)
plot_efc = plot_efc.dropna(subset=['efc'])

# Compute mean EFC per subject across positions
df_efc_mean = plot_efc.groupby(
    ['subject', 'mask_region_label', 'reconstruction'], as_index=False
)['efc'].mean()

# Ensure categorical ordering
recon_order = ['woBin', 'full', '50p', '75p', '95p']
df_efc_mean['reconstruction'] = pd.Categorical(df_efc_mean['reconstruction'], categories=recon_order, ordered=True)
df_efc_mean['mask_region_label'] = pd.Categorical(df_efc_mean['mask_region_label'], categories=['Brain'], ordered=True)

# Boxplot of averaged EFC per subject
fig_avg = px.box(
    df_efc_mean,
    x='mask_region_label',
    y='efc',
    color='reconstruction',
    points='all',
    title='Mean EFC per Subject across Positions',
    category_orders={'mask_region_label': ['Brain'], 'reconstruction': recon_order}
)
fig_avg.update_layout(boxmode='group')
fig_avg.show()


In [16]:
# create a dataframe with efc for masks brain and eyes from merged dataframe to see them together in two separate columns
efc_brain = merged[merged['mask'] == 'brain'][['subject', 'position', 'reconstruction', 'efc']].rename(columns={'efc': 'efc_brain'})
efc_eyes = merged[merged['mask'] == 'eyes'][['subject', 'position', 'reconstruction', 'efc']].rename(columns={'efc': 'efc_eyes'})
efc_combined = pd.merge(efc_brain, efc_eyes, on=['subject', 'position', 'reconstruction'], how='inner')
print(efc_combined.head())
efc_combined.info()


   subject position reconstruction  efc_brain  efc_eyes
0  sub-001     down            50p     0.4335    0.4335
1  sub-001     down            75p     0.4336    0.4336
2  sub-001     down            95p     0.4318    0.4318
3  sub-001     down           full     0.4331    0.4331
4  sub-001     left            50p     0.4327    0.4327
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   subject         51 non-null     object 
 1   position        51 non-null     object 
 2   reconstruction  51 non-null     object 
 3   efc_brain       51 non-null     float64
 4   efc_eyes        51 non-null     float64
dtypes: float64(2), object(3)
memory usage: 2.1+ KB


FBER

https://mriqc.readthedocs.io/en/latest/iqms/t1w.html#mriqc.qc.anatomical.fber

Foreground–background energy ratio

In [17]:
# Build fber-long dataframe from `merged` (mask-level, not tissue-dependent)
records_fber = []
for _, r in merged.iterrows():
    mask = str(r.get('mask')).lower() if r.get('mask') is not None else ''
    if mask not in ('brain', 'eyes'):
        continue
    val = r.get('fber') if 'fber' in r.index else pd.NA
    records_fber.append({
        'subject': r.get('subject'),
        'position': r.get('position'),
        'reconstruction': r.get('reconstruction'),
        'mask': mask,
        'mask_region_label': 'Brain' if mask == 'brain' else 'Eyes',
        'fber': val,
        'bids_name': r.get('bids_name')
    })

plot_fber = pd.DataFrame.from_records(records_fber)
plot_fber = plot_fber.dropna(subset=['fber'])

# Ensure categorical ordering (reuse recon_order and position_order defined earlier)
plot_fber['reconstruction'] = pd.Categorical(plot_fber['reconstruction'], categories=recon_order, ordered=True)
plot_fber['mask_region_label'] = pd.Categorical(plot_fber['mask_region_label'], categories=['Brain', 'Eyes'], ordered=True)
plot_fber['position'] = pd.Categorical(plot_fber['position'], categories=position_order, ordered=True)

# Boxplot of FBER
fig = px.box(
    plot_fber,
    x='mask_region_label',
    y='fber',
    color='reconstruction',
    facet_col='position',
    points='all',
    title='FBER by Mask Type and Position',
    category_orders={
        'mask_region_label': ['Brain', 'Eyes'],
        'reconstruction': recon_order,
        'position': position_order
    }
)
fig.update_layout(boxmode='group')
fig.show()


FBER mean

In [18]:
import pandas as pd
import plotly.express as px

# Assuming 'merged' dataframe is already loaded

# Build fber-long dataframe (brain only)
records_fber = []
for _, r in merged.iterrows():
    mask = str(r.get('mask')).lower() if r.get('mask') is not None else ''
    if mask != 'brain':
        continue
    val = r.get('fber') if 'fber' in r.index else pd.NA
    records_fber.append({
        'subject': r.get('subject'),
        'position': r.get('position'),
        'reconstruction': r.get('reconstruction'),
        'mask': mask,
        'mask_region_label': 'Brain',
        'fber': val,
        'bids_name': r.get('bids_name')
    })

plot_fber = pd.DataFrame.from_records(records_fber)
plot_fber = plot_fber.dropna(subset=['fber'])

# Compute mean FBER per subject across positions (eyes removed)
df_fber_mean = plot_fber.groupby(
    ['subject', 'mask_region_label', 'reconstruction'], as_index=False
)['fber'].mean()

# Ensure categorical ordering
recon_order = ['woBin', 'full', '50p', '75p', '95p']
mask_order_fber = ['Brain']
df_fber_mean['reconstruction'] = pd.Categorical(df_fber_mean['reconstruction'], categories=recon_order, ordered=True)
df_fber_mean['mask_region_label'] = pd.Categorical(df_fber_mean['mask_region_label'], categories=mask_order_fber, ordered=True)

# Boxplot of averaged FBER per subject (eyes removed)
fig_avg = px.box(
    df_fber_mean,
    x='mask_region_label',
    y='fber',
    color='reconstruction',
    points='all',
    title='Mean FBER per Subject across Positions',
    category_orders={'mask_region_label': mask_order_fber, 'reconstruction': recon_order}
)
fig_avg.update_layout(boxmode='group')
fig_avg.show()


art_qi2

https://mriqc.readthedocs.io/en/latest/iqms/t1w.html#mriqc.qc.anatomical.art_qi2

χ² goodness-of-fit on air mask

In [19]:
# Build qi_2-long dataframe from `merged` (mask-level, not tissue-dependent)
records_qi2 = []
for _, r in merged.iterrows():
    mask = str(r.get('mask')).lower() if r.get('mask') is not None else ''
    if mask not in ('brain', 'eyes'):
        continue
    val = r.get('qi_2') if 'qi_2' in r.index else pd.NA
    records_qi2.append({
        'subject': r.get('subject'),
        'position': r.get('position'),
        'reconstruction': r.get('reconstruction'),
        'mask': mask,
        'mask_region_label': 'Brain' if mask == 'brain' else 'Eyes',
        'qi_2': val,
        'bids_name': r.get('bids_name')
    })

plot_qi2 = pd.DataFrame.from_records(records_qi2)
plot_qi2 = plot_qi2.dropna(subset=['qi_2'])

# Ensure categorical ordering (reuse recon_order and position_order defined earlier)
plot_qi2['reconstruction'] = pd.Categorical(plot_qi2['reconstruction'], categories=recon_order, ordered=True)
plot_qi2['mask_region_label'] = pd.Categorical(plot_qi2['mask_region_label'], categories=['Brain', 'Eyes'], ordered=True)
plot_qi2['position'] = pd.Categorical(plot_qi2['position'], categories=position_order, ordered=True)

# Boxplot of QI2
fig = px.box(
    plot_qi2,
    x='mask_region_label',
    y='qi_2',
    color='reconstruction',
    facet_col='position',
    points='all',
    title='QI2 by Mask Type and Position',
    category_orders={
        'mask_region_label': ['Brain', 'Eyes'],
        'reconstruction': recon_order,
        'position': position_order
    }
)
fig.update_layout(boxmode='group')
fig.show()


art_qi2 mean

In [20]:
import pandas as pd
import plotly.express as px

# Assuming 'merged' dataframe is already loaded

# Build qi_2-long dataframe (brain-only, exclude eyes)
records_qi2 = []
for _, r in merged.iterrows():
    mask = str(r.get('mask')).lower() if r.get('mask') is not None else ''
    if mask != 'brain':
        continue
    val = r.get('qi_2') if 'qi_2' in r.index else pd.NA
    records_qi2.append({
        'subject': r.get('subject'),
        'position': r.get('position'),
        'reconstruction': r.get('reconstruction'),
        'mask': mask,
        'mask_region_label': 'Brain',
        'qi_2': val,
        'bids_name': r.get('bids_name')
    })

plot_qi2 = pd.DataFrame.from_records(records_qi2)
plot_qi2 = plot_qi2.dropna(subset=['qi_2'])

# Compute mean QI2 per subject across positions
df_qi2_mean = plot_qi2.groupby(
    ['subject', 'mask_region_label', 'reconstruction'], as_index=False
)['qi_2'].mean()

# Ensure categorical ordering
recon_order = ['woBin', 'full', '50p', '75p', '95p']
df_qi2_mean['reconstruction'] = pd.Categorical(df_qi2_mean['reconstruction'], categories=recon_order, ordered=True)
df_qi2_mean['mask_region_label'] = pd.Categorical(df_qi2_mean['mask_region_label'], categories=['Brain'], ordered=True)

# Boxplot of averaged QI2 per subject
fig_avg = px.box(
    df_qi2_mean,
    x='mask_region_label',
    y='qi_2',
    color='reconstruction',
    points='all',
    title='Mean QI2 per Subject across Positions (brain-only)',
    category_orders={'mask_region_label': ['Brain'], 'reconstruction': recon_order}
)
fig_avg.update_layout(boxmode='group')
fig_avg.show()


FWHM

https://mriqc.readthedocs.io/en/latest/iqms/t1w.html#other-measures

Full-width half-maximum. Image smoothness (blur) measure. Lower values are better, higher values indicate a blurrier image.

In [21]:
# Build fwhm_avg-long dataframe from `merged` (mask-level, not tissue-dependent)
records_fwhm = []
for _, r in merged.iterrows():
    mask = str(r.get('mask')).lower() if r.get('mask') is not None else ''
    if mask not in ('brain', 'eyes'):
        continue
    val = r.get('fwhm_avg') if 'fwhm_avg' in r.index else pd.NA
    records_fwhm.append({
        'subject': r.get('subject'),
        'position': r.get('position'),
        'reconstruction': r.get('reconstruction'),
        'mask': mask,
        'mask_region_label': 'Brain' if mask == 'brain' else 'Eyes',
        'fwhm_avg': val,
        'bids_name': r.get('bids_name')
    })

plot_fwhm = pd.DataFrame.from_records(records_fwhm)
plot_fwhm = plot_fwhm.dropna(subset=['fwhm_avg'])

# Ensure categorical ordering (reuse recon_order and position_order defined earlier)
plot_fwhm['reconstruction'] = pd.Categorical(plot_fwhm['reconstruction'], categories=recon_order, ordered=True)
plot_fwhm['mask_region_label'] = pd.Categorical(plot_fwhm['mask_region_label'], categories=['Brain', 'Eyes'], ordered=True)
plot_fwhm['position'] = pd.Categorical(plot_fwhm['position'], categories=position_order, ordered=True)

# Boxplot of FWHM (average)
fig = px.box(
    plot_fwhm,
    x='mask_region_label',
    y='fwhm_avg',
    color='reconstruction',
    facet_col='position',
    points='all',
    title='FWHM (avg) by Mask Type and Position',
    category_orders={
        'mask_region_label': ['Brain', 'Eyes'],
        'reconstruction': recon_order,
        'position': position_order
    }
)
fig.update_layout(boxmode='group')
fig.show()


FWHM mean

In [22]:
import pandas as pd
import plotly.express as px

# Assuming 'merged' dataframe is already loaded

# Build fwhm_avg-long dataframe (brain-only, exclude eyes)
records_fwhm = []
for _, r in merged.iterrows():
    mask = str(r.get('mask')).lower() if r.get('mask') is not None else ''
    if mask != 'brain':
        continue
    val = r.get('fwhm_avg') if 'fwhm_avg' in r.index else pd.NA
    records_fwhm.append({
        'subject': r.get('subject'),
        'position': r.get('position'),
        'reconstruction': r.get('reconstruction'),
        'mask': mask,
        'mask_region_label': 'Brain',
        'fwhm_avg': val,
        'bids_name': r.get('bids_name')
    })

plot_fwhm = pd.DataFrame.from_records(records_fwhm)
plot_fwhm = plot_fwhm.dropna(subset=['fwhm_avg'])

# Compute mean FWHM per subject across positions
df_fwhm_mean = plot_fwhm.groupby(
    ['subject', 'mask_region_label', 'reconstruction'], as_index=False
)['fwhm_avg'].mean()

# Ensure categorical ordering
recon_order = ['woBin', 'full', '50p', '75p', '95p']
df_fwhm_mean['reconstruction'] = pd.Categorical(df_fwhm_mean['reconstruction'], categories=recon_order, ordered=True)
df_fwhm_mean['mask_region_label'] = pd.Categorical(df_fwhm_mean['mask_region_label'], categories=['Brain'], ordered=True)

# Boxplot of averaged FWHM per subject
fig_avg = px.box(
    df_fwhm_mean,
    x='mask_region_label',
    y='fwhm_avg',
    color='reconstruction',
    points='all',
    title='Mean FWHM per Subject across Positions',
    category_orders={'mask_region_label': ['Brain'], 'reconstruction': recon_order}
)
fig_avg.update_layout(boxmode='group')
fig_avg.show()


CNR mean

https://mriqc.readthedocs.io/en/latest/iqms/t1w.html#mriqc.qc.anatomical.cnr

Calculate the CNR [Magnota2006]. Higher values are better.

In [23]:
import pandas as pd
import plotly.express as px

# Assuming 'merged' dataframe is already loaded

# Build cnr-long dataframe (brain-only, exclude eyes)
records_cnr = []
for _, r in merged.iterrows():
    mask = str(r.get('mask')).lower() if r.get('mask') is not None else ''
    if mask != 'brain':
        continue
    val = r.get('cnr') if 'cnr' in r.index else pd.NA
    records_cnr.append({
        'subject': r.get('subject'),
        'position': r.get('position'),
        'reconstruction': r.get('reconstruction'),
        'mask': mask,
        'mask_region_label': 'Brain',
        'cnr': val,
        'bids_name': r.get('bids_name')
    })

plot_cnr = pd.DataFrame.from_records(records_cnr)
plot_cnr = plot_cnr.dropna(subset=['cnr'])

# Compute mean CNR per subject across positions
df_cnr_mean = plot_cnr.groupby(
    ['subject', 'mask_region_label', 'reconstruction'], as_index=False
)['cnr'].mean()

# Ensure categorical ordering
recon_order = ['woBin', 'full', '50p', '75p', '95p']
df_cnr_mean['reconstruction'] = pd.Categorical(df_cnr_mean['reconstruction'], categories=recon_order, ordered=True)
df_cnr_mean['mask_region_label'] = pd.Categorical(df_cnr_mean['mask_region_label'], categories=['Brain'], ordered=True)

# Boxplot of averaged CNR per subject
fig_avg = px.box(
    df_cnr_mean,
    x='mask_region_label',
    y='cnr',
    color='reconstruction',
    points='all',
    title='Mean CNR per Subject across Positions',
    category_orders={'mask_region_label': ['Brain'], 'reconstruction': recon_order}
)
fig_avg.update_layout(boxmode='group')
fig_avg.show()


CJV mean

https://mriqc.readthedocs.io/en/latest/iqms/t1w.html#mriqc.qc.anatomical.cjv

Calculate the CJV, a measure related to SNR and CNR that is presented as a proxy for the INU artifact [Ganzetti2016]. Lower is better.

In [24]:
import pandas as pd
import plotly.express as px

# Assuming 'merged' dataframe is already loaded

# Build cjv-long dataframe (brain-only, exclude eyes)
records_cjv = []
for _, r in merged.iterrows():
    mask = str(r.get('mask')).lower() if r.get('mask') is not None else ''
    if mask != 'brain':
        continue
    val = r.get('cjv') if 'cjv' in r.index else pd.NA
    records_cjv.append({
        'subject': r.get('subject'),
        'position': r.get('position'),
        'reconstruction': r.get('reconstruction'),
        'mask': mask,
        'mask_region_label': 'Brain',
        'cjv': val,
        'bids_name': r.get('bids_name')
    })

plot_cjv = pd.DataFrame.from_records(records_cjv)
plot_cjv = plot_cjv.dropna(subset=['cjv'])

# Compute mean CJV per subject across positions
df_cjv_mean = plot_cjv.groupby(
    ['subject', 'mask_region_label', 'reconstruction'], as_index=False
)['cjv'].mean()

# Ensure categorical ordering
recon_order = ['woBin', 'full', '50p', '75p', '95p']
df_cjv_mean['reconstruction'] = pd.Categorical(df_cjv_mean['reconstruction'], categories=recon_order, ordered=True)
df_cjv_mean['mask_region_label'] = pd.Categorical(df_cjv_mean['mask_region_label'], categories=['Brain'], ordered=True)

# Boxplot of averaged CJV per subject
fig_avg = px.box(
    df_cjv_mean,
    x='mask_region_label',
    y='cjv',
    color='reconstruction',
    points='all',
    title='Mean CJV per Subject across Positions (brain-only)',
    category_orders={'mask_region_label': ['Brain'], 'reconstruction': recon_order}
)
fig_avg.update_layout(boxmode='group')
fig_avg.show()


wm2max

In [25]:
import pandas as pd
import plotly.express as px

# Assuming 'merged' dataframe is already loaded

# Build wm2max-long dataframe (brain-only, exclude eyes)
records_wm2max = []
for _, r in merged.iterrows():
    mask = str(r.get('mask')).lower() if r.get('mask') is not None else ''
    if mask != 'brain':
        continue
    val = r.get('wm2max') if 'wm2max' in r.index else pd.NA
    records_wm2max.append({
        'subject': r.get('subject'),
        'position': r.get('position'),
        'reconstruction': r.get('reconstruction'),
        'mask': mask,
        'mask_region_label': 'Brain',
        'wm2max': val,
        'bids_name': r.get('bids_name')
    })

plot_wm2max = pd.DataFrame.from_records(records_wm2max)
plot_wm2max = plot_wm2max.dropna(subset=['wm2max'])

# Compute mean wm2max per subject across positions
df_wm2max_mean = plot_wm2max.groupby(
    ['subject', 'mask_region_label', 'reconstruction'], as_index=False
)['wm2max'].mean()

# Ensure categorical ordering
recon_order = ['woBin', 'full', '50p', '75p', '95p']
df_wm2max_mean['reconstruction'] = pd.Categorical(df_wm2max_mean['reconstruction'], categories=recon_order, ordered=True)
df_wm2max_mean['mask_region_label'] = pd.Categorical(df_wm2max_mean['mask_region_label'], categories=['Brain'], ordered=True)

# Boxplot of averaged wm2max per subject
fig_avg = px.box(
    df_wm2max_mean,
    x='mask_region_label',
    y='wm2max',
    color='reconstruction',
    points='all',
    title='Mean WM2MAX per Subject across Positions (brain-only)',
    category_orders={'mask_region_label': ['Brain'], 'reconstruction': recon_order}
)
fig_avg.update_layout(boxmode='group')
fig_avg.show()


## Line plots

Brain IQMs

In [60]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Metrics for Figure 1 (non-tissue-specific)
metrics_fig1 = [
    (df_cjv_mean, 'cjv', 'CJV'),
    (df_cnr_mean, 'cnr', 'CNR'),
    (df_efc_mean, 'efc', 'EFC'),
    (df_fber_mean, 'fber', 'FBER'),
    (df_fwhm_mean, 'fwhm_avg', 'FWHM'),
    (df_qi2_mean, 'qi_2', 'QI2'),
    (df_inu_mean, 'inu_med', 'INU'),
    (df_wm2max_mean, 'wm2max', 'WM2MAX')
]

recon_order = ['woBin', 'full', '50p', '75p', '95p']

# Create subplots: 2 rows x 4 columns
fig1 = make_subplots(
    rows=2, cols=4,
    subplot_titles=[name for _, _, name in metrics_fig1]
)

subjects = df_cjv_mean['subject'].unique()
color_map = {subj: color for subj, color in zip(subjects, ['red', 'blue', 'green'])}

# Function to add traces for non-tissue metrics
def add_traces_general(fig, df, col, row, y_col):
    for subj in subjects:
        subj_df = df[df['subject'] == subj]
        fig.add_trace(
            go.Scatter(
                x=subj_df['reconstruction'],
                y=subj_df[y_col],
                mode='lines+markers',
                name=f'Subject {subj}',
                line=dict(color=color_map[subj]),
                marker=dict(color=color_map[subj]),
                showlegend=(col==1 and row==1)
            ),
            row=row, col=col
        )

# Add all metrics to fig1
for idx, (df, y_col, _) in enumerate(metrics_fig1):
    row = idx // 4 + 1
    col = idx % 4 + 1
    add_traces_general(fig1, df, col, row, y_col)

fig1.update_layout(
    title_text="IQMs per subject across reconstructions",
    height=800,
    width=1400
)
fig1.show()


SNR, SNRd

In [61]:
# Metrics for Figure 2 (tissue-specific)
metrics_fig2 = [
    (df_snr_mean, 'snr', 'SNR'),
    (df_snrd_mean, 'snrd', 'SNRD')
]

tissue_order = ['Whole Volume', 'Brain', 'CSF', 'GM', 'WM']

fig2 = make_subplots(
    rows=1, cols=2,
    subplot_titles=[name for _, _, name in metrics_fig2]
)

def add_traces_tissue_ordered(fig, df, col, row, y_col):
    for subj in subjects:
        subj_df = df[df['subject'] == subj]
        # Order by tissue first, then reconstruction
        x_vals = []
        y_vals = []
        for tissue in tissue_order:
            tissue_df = subj_df[subj_df['mask_region_label'] == tissue]
            # Sort by reconstruction
            tissue_df = tissue_df.sort_values('reconstruction')
            x_vals.extend([f"{rec}-{tissue}" for rec in tissue_df['reconstruction']])
            y_vals.extend(tissue_df[y_col].tolist())
        fig.add_trace(
            go.Scatter(
                x=x_vals,
                y=y_vals,
                mode='lines+markers',
                name=f'Subject {subj}',
                line=dict(color=color_map[subj]),
                marker=dict(color=color_map[subj]),
                showlegend=(col==1 and row==1)
            ),
            row=row, col=col
        )

# Add SNR and SNRD traces
for idx, (df, y_col, _) in enumerate(metrics_fig2):
    add_traces_tissue_ordered(fig2, df, col=idx+1, row=1, y_col=y_col)

fig2.update_layout(
    title_text="Tissue-specific Metrics per Subject across Reconstructions and Regions",
    height=500,
    width=1200
)
fig2.show()


## mean+-std tables

IQMs

In [55]:
import pandas as pd
import numpy as np

# Define the metric dataframes and column names (as in your plotting code)
metrics_fig1 = [
    (df_cjv_mean, 'cjv', 'CJV'),
    (df_cnr_mean, 'cnr', 'CNR'),
    (df_efc_mean, 'efc', 'EFC'),
    (df_fber_mean, 'fber', 'FBER'),
    (df_fwhm_mean, 'fwhm_avg', 'FWHM'),
    (df_qi2_mean, 'qi_2', 'QI2'),
    (df_inu_mean, 'inu_med', 'INU'),
    (df_wm2max_mean, 'wm2max', 'WM2MAX')
]

recon_order = ['woBin', 'full', '50p', '75p', '95p']

summary_tables = []

for df, y_col, metric_name in metrics_fig1:
    # Compute mean and std by reconstruction
    summary = df.groupby(['reconstruction'], observed=True)[y_col].agg(['mean', 'std']).reset_index()
    if summary.empty:
        continue

    # Ensure consistent ordering for lookup
    summary['reconstruction'] = pd.Categorical(summary['reconstruction'], categories=recon_order, ordered=True)
    summary = summary.sort_values('reconstruction')

    # Reference to 'full' (safe fallback if missing or zero)
    full_mean_series = summary.loc[summary['reconstruction'] == 'full', 'mean']
    if not full_mean_series.empty and full_mean_series.values[0] != 0:
        full_mean = full_mean_series.values[0]
    else:
        full_mean = summary['mean'].iloc[0] if not summary['mean'].empty else np.nan

    # Compute percent change (guard divide-by-zero)
    if pd.isna(full_mean) or full_mean == 0:
        summary['perc_change'] = pd.NA
    else:
        summary['perc_change'] = (summary['mean'] - full_mean) / full_mean * 100

    # Create formatted string
    summary['formatted'] = summary.apply(
        lambda row: f"{row['mean']:.4f} ± {row['std']:.4f} ({row['perc_change']:+.2f}%)" 
        if pd.notna(row['perc_change']) else f"{row['mean']:.4f} ± {row['std']:.4f}",
        axis=1
    )

    # Build a single-row table for this metric with one column per reconstruction
    row = {'metric': metric_name}
    for col in recon_order:
        match = summary.loc[summary['reconstruction'] == col, 'formatted']
        row[col] = match.values[0] if not match.empty else ""

    # add region column with value 'Brain'
    row['region'] = 'Brain'

    table = pd.DataFrame([row])
    summary_tables.append(table)

# Concatenate all metrics vertically (one row per metric)
if summary_tables:
    final_summary = pd.concat(summary_tables, ignore_index=True)
    # Ensure column order: metric, region, then reconstructions
    cols = ['metric', 'region'] + [c for c in final_summary.columns if c not in ('metric', 'region')]
    final_summary = final_summary[cols]
else:
    final_summary = pd.DataFrame(columns=['metric', 'region'] + recon_order)

# Display nicely
pd.set_option('display.max_colwidth', None)

# Save table as csv
final_summary.to_csv("iqms_summary_table_1.csv", index=False)

final_summary


,metric,region,woBin,full,50p,75p,95p
0,CJV,Brain,2.3589 ± 0.4524 (-14.64%),2.7635 ± 0.4351 (+0.00%),2.7565 ± 0.3658 (-0.25%),2.8617 ± 0.4233 (+3.55%),3.6259 ± 0.6163 (+31.21%)
1,CNR,Brain,0.5104 ± 0.0560 (+12.81%),0.4524 ± 0.0503 (+0.00%),0.4514 ± 0.0440 (-0.22%),0.4362 ± 0.0599 (-3.59%),0.3598 ± 0.0763 (-20.48%)
2,EFC,Brain,0.4406 ± 0.0429 (-1.58%),0.4476 ± 0.0433 (+0.00%),0.4477 ± 0.0433 (+0.02%),0.4477 ± 0.0452 (+0.02%),0.4503 ± 0.0484 (+0.59%)
3,FBER,Brain,5157.7041 ± 3627.8563 (+34.36%),3838.8224 ± 2703.3698 (+0.00%),3794.6201 ± 2667.7519 (-1.15%),3859.2159 ± 2732.6444 (+0.53%),2923.2472 ± 1753.0879 (-23.85%)
4,FWHM,Brain,5.7073 ± 0.6282 (+9.66%),5.2043 ± 0.5092 (+0.00%),5.2683 ± 0.4630 (+1.23%),5.4944 ± 0.4267 (+5.57%),6.3747 ± 0.3529 (+22.49%)
5,QI2,Brain,0.0004 ± 0.0000 (+7.45%),0.0004 ± 0.0001 (+0.00%),0.0004 ± 0.0001 (-2.24%),0.0004 ± 0.0001 (-2.63%),0.0004 ± 0.0002 (-4.53%)
6,INU,Brain,0.2444 ± 0.0398 (-9.12%),0.2689 ± 0.0416 (+0.00%),0.2686 ± 0.0416 (-0.11%),0.2689 ± 0.0475 (-0.02%),0.2762 ± 0.0642 (+2.70%)
7,WM2MAX,Brain,0.9069 ± 0.0065 (+2.16%),0.8877 ± 0.0059 (+0.00%),0.8869 ± 0.0065 (-0.10%),0.8872 ± 0.0073 (-0.06%),0.8792 ± 0.0082 (-0.96%)


SNR, SNRd

In [50]:
# Summarize tissue-specific metrics (mean ± std and % change vs 'full') for SNR and SNRD
import pandas as pd
import numpy as np

recon_order = ['woBin', 'full', '50p', '75p', '95p']
tissue_order = ['Whole Volume', 'Brain', 'CSF', 'GM', 'WM']  # include Whole Volume for completeness if present

def summarize_tissue_metric(df, value_col, metric_name, recon_order=recon_order, tissue_order=tissue_order):
    tables = []
    for tissue in tissue_order:
        tissue_df = df[df['mask_region_label'] == tissue]
        if tissue_df.empty:
            continue

        summary = tissue_df.groupby(['reconstruction'], observed=True)[value_col].agg(['mean', 'std']).reset_index()
        if summary.empty:
            continue

        summary['reconstruction'] = pd.Categorical(summary['reconstruction'], categories=recon_order, ordered=True)
        summary = summary.sort_values('reconstruction').reset_index(drop=True)

        # reference value: 'full' if present and non-zero, else first available mean
        full_mean_series = summary.loc[summary['reconstruction'] == 'full', 'mean']
        if not full_mean_series.empty and full_mean_series.values[0] != 0:
            full_mean = full_mean_series.values[0]
        else:
            full_mean = summary['mean'].iloc[0] if not summary['mean'].empty else np.nan

        if pd.isna(full_mean) or full_mean == 0:
            summary['perc_change'] = pd.NA
        else:
            summary['perc_change'] = (summary['mean'] - full_mean) / full_mean * 100

        summary['formatted'] = summary.apply(
            lambda row: f"{row['mean']:.4f} ± {row['std']:.4f} ({row['perc_change']:+.2f}%)"
            if pd.notna(row['perc_change']) else f"{row['mean']:.4f} ± {row['std']:.4f}",
            axis=1
        )

        # create row with separate region column
        row = {'metric': metric_name, 'region': tissue}
        for col in recon_order:
            match = summary.loc[summary['reconstruction'] == col, 'formatted']
            row[col] = match.values[0] if not match.empty else ""
        tables.append(pd.DataFrame([row]))

    # ensure consistent columns even if empty
    cols = ['metric', 'region'] + recon_order
    if tables:
        result = pd.concat(tables, ignore_index=True)
        # reindex to ensure column order
        return result.reindex(columns=cols)
    else:
        return pd.DataFrame(columns=cols)

# Generate summaries
snr_summary = summarize_tissue_metric(df_snr_mean, 'snr', 'SNR')
snrd_summary = summarize_tissue_metric(df_snrd_mean, 'snrd', 'SNRD')

# Combine or display separately
pd.set_option('display.max_colwidth', None)
final_snr_snrd = pd.concat([snr_summary, snrd_summary], ignore_index=True)

# Save table as csv
final_snr_snrd.to_csv("iqms_summary_table_2.csv", index=False)

final_snr_snrd


,metric,region,woBin,full,50p,75p,95p
0,SNR,Brain,15.8842 ± 0.6238 (+8.16%),14.6858 ± 0.3791 (+0.00%),14.6368 ± 0.3923 (-0.33%),14.8329 ± 0.1788 (+1.00%),15.0611 ± 0.5069 (+2.56%)
1,SNR,CSF,2.9166 ± 0.2186 (-1.97%),2.9752 ± 0.2139 (+0.00%),3.0039 ± 0.2082 (+0.96%),3.0767 ± 0.1841 (+3.41%),3.6276 ± 0.2410 (+21.92%)
2,SNR,GM,13.0618 ± 0.7871 (+1.95%),12.8120 ± 0.5612 (+0.00%),12.8480 ± 0.4588 (+0.28%),13.2528 ± 0.3168 (+3.44%),14.3039 ± 0.6950 (+11.64%)
3,SNR,WM,31.6742 ± 1.1360 (+12.04%),28.2701 ± 0.5963 (+0.00%),28.0586 ± 0.7538 (-0.75%),28.1694 ± 0.4838 (-0.36%),27.2518 ± 0.8720 (-3.60%)
4,SNRD,Brain,49.3678 ± 18.7072 (+12.25%),43.9815 ± 15.3960 (+0.00%),43.7360 ± 15.0923 (-0.56%),46.5984 ± 18.5283 (+5.95%),43.9378 ± 17.7224 (-0.10%)
5,SNRD,CSF,39.0203 ± 15.5128 (+10.44%),35.3301 ± 12.8315 (+0.00%),35.2276 ± 12.6234 (-0.29%),37.7892 ± 15.0999 (+6.96%),37.1379 ± 14.7714 (+5.12%)
6,SNRD,GM,53.3499 ± 19.9059 (+12.75%),47.3187 ± 16.3470 (+0.00%),47.0134 ± 16.0095 (-0.65%),50.0009 ± 19.7961 (+5.67%),46.5870 ± 18.8383 (-1.55%)
7,SNRD,WM,55.7333 ± 20.7066 (+13.06%),49.2956 ± 17.0189 (+0.00%),48.9671 ± 16.6480 (-0.67%),52.0050 ± 20.6950 (+5.50%),48.0884 ± 19.5693 (-2.45%)


Merged table with all metrics (SSIM, DCTE, IQMs)

In [58]:
import pandas as pd

# Paths
ssim_csv = '/home/debi/jaime/repos/MR-EyeTrack/analysis/comparison/ssim/ssim_summary_table.csv'
dcte_csv = '/home/debi/jaime/repos/MR-EyeTrack/analysis/comparison/dcte/dcte_summary_table.csv'
iqms1_csv = '/home/debi/jaime/repos/MR-EyeTrack/analysis/iqms_summary_table_1.csv'
iqms2_csv = '/home/debi/jaime/repos/MR-EyeTrack/analysis/iqms_summary_table_2.csv'

# Read all CSVs
dfs = [pd.read_csv(path) for path in [ssim_csv, dcte_csv, iqms1_csv, iqms2_csv]]

# Concatenate vertically
df_merged = pd.concat(dfs, ignore_index=True)

# Save df_merged to csv
df_merged.to_csv('merged_all_metrics.csv')

# Optional: verify shape and preview
print(df_merged.shape)
print(df_merged.head())


(28, 7)
  metric region                       50p                       75p  \
0   SSIM  Brain  0.9298 ± 0.0158 (-0.37%)  0.8788 ± 0.0260 (-5.84%)   
1   SSIM    CSF  0.9327 ± 0.0159 (-0.67%)  0.8711 ± 0.0300 (-7.24%)   
2   SSIM     GM  0.9245 ± 0.0180 (-0.46%)  0.8668 ± 0.0325 (-6.68%)   
3   SSIM     WM  0.9331 ± 0.0181 (-1.29%)  0.8901 ± 0.0304 (-5.84%)   
4   SSIM   Eyes  0.9026 ± 0.0299 (+1.68%)  0.8421 ± 0.0375 (-5.14%)   

                         95p                     woBin full  
0  0.7906 ± 0.0299 (-15.28%)  0.9333 ± 0.0149 (+0.00%)  NaN  
1  0.7469 ± 0.0410 (-20.47%)  0.9391 ± 0.0130 (+0.00%)  NaN  
2  0.7578 ± 0.0422 (-18.41%)  0.9288 ± 0.0170 (+0.00%)  NaN  
3  0.8229 ± 0.0343 (-12.95%)  0.9453 ± 0.0150 (+0.00%)  NaN  
4  0.7265 ± 0.0350 (-18.16%)  0.8877 ± 0.0376 (+0.00%)  NaN  
